In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
cdphDeficiency = pd.read_csv('CDPH-DEF.csv')

In [4]:
cdssDeficiency = pd.read_csv('CDSS-def.csv')

In [5]:
cdphFac = pd.read_csv('cdph-facilities.csv')

In [6]:
cdssFac = pd.read_csv('cdss-facilities.csv')

In [7]:
snfOutbreaks = pd.read_csv('CA_DETAIL_TABLE_crosstab-8-2-20.csv')

In [8]:
snfOutbreaks['Facility Id'] = snfOutbreaks['Facility Id'].apply(lambda x: '{0:0>10}'.format(x))

In [9]:
snfOutbreaks['CAFACID'] = 'CA' + snfOutbreaks['Facility Id'].astype(str)

In [10]:
cdphFacDef = cdphFac.merge(cdphDeficiency, on='CAFACID', how='left')

In [11]:
cdphAll = cdphFacDef.merge(snfOutbreaks, on='CAFACID', how='left')

In [12]:
cdphAll.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23145 entries, 0 to 23144
Data columns (total 90 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   FACID                          23145 non-null  int64  
 1   CAFACID                        23145 non-null  object 
 2   NPI                            18219 non-null  object 
 3   FACNAME                        23145 non-null  object 
 4   FAC_TYPE_C                     23145 non-null  object 
 5   FAC_FDR_x                      23145 non-null  object 
 6   LTC_x                          23145 non-null  object 
 7   CAPACITY                       23145 non-null  int64  
 8   ADDRESS_x                      23145 non-null  object 
 9   CITY                           23145 non-null  object 
 10  ZIP                            23145 non-null  int64  
 11  ZIP9                           20839 non-null  float64
 12  FACADMIN                       20539 non-null 

First, adding a null value to CDPH regulated facilities without violations and adding a flag on if there are any. Doing the same for skilled nursing homes with outbreaks.

In [13]:
cdphAll['Count of fac violations'] = cdphAll['Count of fac violations'].fillna(0)
cdphAll['any violations'] = cdphAll.apply(lambda x: True if x['Count of fac violations']>0 else False, axis=1)

In [14]:
cdphAll['CUMULATIVE POSITIVE RESIDENTS'].value_counts()

Series([], Name: CUMULATIVE POSITIVE RESIDENTS, dtype: int64)

In [15]:
cdphAll['CUMULATIVE POSITIVE RESIDENTS'] = cdphAll['CUMULATIVE POSITIVE RESIDENTS'].fillna('0')

In [16]:
cdphAll['covidres'] = ["None" if ele == '0' else "Some" for ele in cdphAll['CUMULATIVE POSITIVE RESIDENTS']]

In [17]:
cdphAll.covidres.value_counts()

None    23145
Name: covidres, dtype: int64

In [18]:
cdphAll.groupby('covidres')['CAFACID'].nunique()

covidres
None    11901
Name: CAFACID, dtype: int64

In [19]:
cdphAll['CUMULATIVE POSITIVE HCW'] = cdphAll['CUMULATIVE POSITIVE HCW'].fillna('0')

In [20]:
cdphAll['covidhcw'] = ["None" if ele == '0' else "Some" for ele in cdphAll['CUMULATIVE POSITIVE HCW']]

In [21]:
cdphAll['covid'] = cdphAll.apply(lambda x: True if x['covidhcw'] == "Some" or x['covidres'] == "Some" else 'None', axis=1)

In [22]:
cdphAll.covid.value_counts()

None    23145
Name: covid, dtype: int64

In [23]:
cdph = cdphAll[['CAFACID','FAC_FDR_x','LTC_x','COUNTY_NAM','WUICLASS10','HAZ_CLASS','Risk','Count of fac violations','list of complaints','list of scp_sev','All reg_id','E flag','CUMULATIVE POSITIVE RESIDENTS','CUMULATIVE POSITIVE HCW','any violations','covidres','covidhcw','covid']]

In [24]:
cdph.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23145 entries, 0 to 23144
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   CAFACID                        23145 non-null  object 
 1   FAC_FDR_x                      23145 non-null  object 
 2   LTC_x                          23145 non-null  object 
 3   COUNTY_NAM                     23145 non-null  object 
 4   WUICLASS10                     23143 non-null  object 
 5   HAZ_CLASS                      23143 non-null  object 
 6   Risk                           23145 non-null  object 
 7   Count of fac violations        23145 non-null  float64
 8   list of complaints             13284 non-null  object 
 9   list of scp_sev                13284 non-null  object 
 10  All reg_id                     13284 non-null  object 
 11  E flag                         984 non-null    object 
 12  CUMULATIVE POSITIVE RESIDENTS  23145 non-null 

View only open facilities. 

In [25]:
cdph_open = cdph.loc[cdph.LTC_x == 'LTC']

Filtering to just the types of facilities we're examining: skilled nursing homes, intermediate care and congregate care facilities.

In [26]:
cdph_open.FAC_FDR_x.value_counts()

SKILLED NURSING FACILITY                        9398
INTERMEDIATE CARE FACILITY-DD/H/N/CN/IID        4167
CONGREGATE LIVING HEALTH FACILITY                239
INTERMEDIATE CARE FACILITY                        18
PEDIATRIC DAY HEALTH & RESPITE CARE FACILITY      18
Name: FAC_FDR_x, dtype: int64

In [27]:
skilled = cdph_open.FAC_FDR_x == 'SKILLED NURSING FACILITY'
inter_care_plus = cdph_open.FAC_FDR_x == 'INTERMEDIATE CARE FACILITY-DD/H/N/CN/IID'
inter_care = cdph_open.FAC_FDR_x == 'INTERMEDIATE CARE FACILITY'
cong_health = cdph_open.FAC_FDR_x == 'CONGREGATE LIVING HEALTH FACILITY'

cdph_open = cdph_open.loc[skilled | inter_care_plus | inter_care | cong_health]

In [28]:
cdph_open.groupby('FAC_FDR_x')['CAFACID'].nunique()

FAC_FDR_x
CONGREGATE LIVING HEALTH FACILITY            239
INTERMEDIATE CARE FACILITY                    10
INTERMEDIATE CARE FACILITY-DD/H/N/CN/IID    1113
SKILLED NURSING FACILITY                    1216
Name: CAFACID, dtype: int64

How many facilities are in risky areas?

In [29]:
cdph_open.groupby('Risk')['CAFACID'].nunique()

Risk
Ok       1818
Risky     760
Name: CAFACID, dtype: int64

In [30]:
pd.pivot_table(cdph_open, index=['FAC_FDR_x','Risk'], values=['CAFACID'], aggfunc=pd.Series.nunique)

CAFACID
FAC_FDR_x                                Risk          
CONGREGATE LIVING HEALTH FACILITY        Ok         140
                                         Risky       99
INTERMEDIATE CARE FACILITY               Ok           6
                                         Risky        4
INTERMEDIATE CARE FACILITY-DD/H/N/CN/IID Ok         758
                                         Risky      355
SKILLED NURSING FACILITY                 Ok         914
                                         Risky      302

In [31]:
snfandint = cdph_open.loc[skilled | inter_care_plus | inter_care]

Did regulators find emergency preparedness or fire deficiencies?

In [32]:
snfandint.groupby('any violations')['CAFACID'].nunique()

any violations
False     299
True     2040
Name: CAFACID, dtype: int64

In [33]:
pd.pivot_table(snfandint, index=['FAC_FDR_x','any violations'], values=['CAFACID'], aggfunc=pd.Series.nunique)

CAFACID
FAC_FDR_x                                any violations         
INTERMEDIATE CARE FACILITY               False                 8
                                         True                  2
INTERMEDIATE CARE FACILITY-DD/H/N/CN/IID False               228
                                         True                885
SKILLED NURSING FACILITY                 False                63
                                         True               1153

How many skilled nursing homes have already had outbreaks?

In [34]:
snfalone = cdph_open.loc[skilled]

In [35]:
snfalone.covid.value_counts()

None    9398
Name: covid, dtype: int64

In [36]:
pd.pivot_table(snfalone, index=['Risk','covid'], values=['CAFACID'], aggfunc=pd.Series.nunique)

,,CAFACID
Risk,covid,
Ok,None,914
Risky,None,302


Let's examine residential care facilities regulated by CDSS.

In [37]:
cdssFacDef = cdssFac.merge(cdssDeficiency, on='Facility Number', how='left') 

In [38]:
cdssFacDef['General section'] = cdssFacDef['General section'].fillna('None')

In [39]:
cdssFacDef['deficient'] = ["None" if ele == 'None' else "Some" for ele in cdssFacDef['General section']]

We only want facilities that are currently in operation.

In [40]:
licensed = cdssFacDef['Facility Status'] == 'LICENSED'
probation = cdssFacDef['Facility Status'] == 'ON PROBATION'

cdss_open = cdssFacDef.loc[licensed | probation]

In [41]:
cdss_open['Facility Status'].value_counts()

LICENSED        7543
ON PROBATION      41
Name: Facility Status, dtype: int64

In [42]:
cdss_open['Facility Type_x'].value_counts()

RESIDENTIAL CARE ELDERLY                     7477
RCFE-CONTINUING CARE RETIREMENT COMMUNITY     107
Name: Facility Type_x, dtype: int64

In [43]:
cdss_open.groupby('deficient')['Facility Number'].nunique()

deficient
None    6876
Some     605
Name: Facility Number, dtype: int64

In [44]:
cdss_open['Risk'].value_counts()

Ok       4809
Risky    2775
Name: Risk, dtype: int64

In [45]:
pd.pivot_table(cdss_open, index=['deficient','Risk'], values=['Facility Number'], aggfunc=pd.Series.nunique)

Facility Number
deficient Risk                  
None      Ok                4416
          Risky             2460
Some      Ok                 330
          Risky              275